# Import Modules

In [1]:
import numpy as np
import pandas as pd
import bs4
import requests
import re
from dotenv import load_dotenv
import os
import sqlalchemy as db
from geopy.geocoders import Nominatim, Photon
import folium

import matplotlib.pyplot as plt
import seaborn as sns
import pydeck as pdk

%matplotlib inline

# Data Wrangling

### Load .env file

In [35]:
load_dotenv()

True

In [3]:
COOKIES = {'FORLAPDIKTI': os.getenv('FORLAPDIKTI'), 
           'PHPSESSID': os.getenv('PHPSESSID'), 
           'FGTServer': os.getenv('ADRUM')}

BASE_URL = 'https://forlap.kemdikbud.go.id/perguruantinggi/search'

### Fetch universities data from web

In [7]:
data = pd.DataFrame()
PAGE = 0

while True:
    response = requests.get(f'https://forlap.kemdikbud.go.id/perguruantinggi/search/{PAGE}', cookies=COOKIES)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    new_table = pd.read_html(response.text, thousands='.')[0]
    data = pd.concat([data, new_table], ignore_index=True)
    
    if soup.select('div.pagination ul li')[-1].find_all('a') == []:
        break
    else:
        PAGE += 20

### Fetch major data of each universites from web (Cannot be done)

In [8]:
data_prodi = pd.DataFrame()
PAGE = 0

while True:
    response = requests.get(f'https://forlap.kemdikbud.go.id/perguruantinggi/search/{PAGE}', cookies=COOKIES)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    list_univ = [(x.select('td')[1].getText().strip(), 
                  x.select('td')[2].getText(),
                  x.select('a')[0]['href']) 
                  for x in soup.select('tr.ttop') if x.select('td')[3].getText() == 'Prov. Kalimantan Utara']
    for kode, nama, link in list_univ:
        response = requests.get(link)
        prodi_univ = pd.read_html(response.text, thousands='.')[-1]
        prodi_univ[('Kode PT', 'Kode PT')] = kode
        prodi_univ[('Nama PT', 'Nama PT')] = nama
        data_prodi = pd.concat([data_prodi, prodi_univ], ignore_index=True)
    
    if soup.select('div.pagination ul li')[-1].find_all('a') == []:
        break
    else:
        PAGE += 20

### Rename columns name and transform dataframe into non-multiindex

In [9]:
first = list(data.columns.get_level_values(1)[:6])
second = list(map(lambda x: x+' 2018/2019', data.columns.get_level_values(1)[6:9]))
third = list(map(lambda x: x+' 2019/2020', data.columns.get_level_values(1)[9:]))

column_names = first + second + third
column_names

['No.',
 'Kode PT',
 'Nama PT',
 'Provinsi',
 'Kategori',
 'Status',
 'Jml Dosen Tetap 2018/2019',
 'Jml Mhs 2018/2019',
 'Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019',
 'Jml Dosen Tetap 2019/2020',
 'Jml Mhs 2019/2020',
 'Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020']

In [10]:
data.columns = data.columns.droplevel(level=0)
data.columns = column_names
data.head(5)

,No.,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
0,1,0,NaN,NaN,NaN,NaN,0,0,1 :,0,0,1 : 0
1,2,1001,Universitas Gadjah Mada,Prov. D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
2,3,1002,Universitas Indonesia,Prov. D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
3,4,1003,Universitas Sumatera Utara,Prov. Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
4,5,1004,Universitas Airlangga,Prov. Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1


### Delete Rows/Columns

In [15]:
# Before deletion
data.shape

(5717, 12)

In [46]:
# After deletion
data.dropna(subset=['Nama PT'], inplace=True)
data.drop(labels='No.', axis=1, inplace=True)
data.shape

(5716, 11)

### Value Reformatting

In [47]:
data.head()

,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
1,1001,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
2,1002,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
3,1003,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
4,1004,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1
5,1005,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2


In [40]:
# Delete `Prov.` in Provinsi column
data['Provinsi'] = data['Provinsi'].str.replace('Prov. ', '')
data.head()

,No.,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
1,2,1001,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
2,3,1002,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
3,4,1003,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
4,5,1004,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1
5,6,1005,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2


# Data Storing

In [4]:
engine = db.create_engine(f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}")

In [70]:
with engine.connect() as conn:
    data.to_sql('universitas', con=conn, index=False)

# Data Fetching from Database

In [5]:
engine = db.create_engine(f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}")

In [6]:
meta = db.MetaData(bind=engine)
meta.reflect()

In [7]:
univ = db.Table('universitas', meta, autoload=True, autoload_with=engine)

In [8]:
query = db.select([univ])
with engine.connect() as conn:
    result = conn.execute(query).fetchall()
len(result)

5716

In [9]:
data = pd.DataFrame.from_records(result, columns=univ.c.keys())
data

,index,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
0,1,1001,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
1,2,1002,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
2,3,1003,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
3,4,1004,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1
4,5,1005,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2
...,...,...,...,...,...,...,...,...,...,...,...,...
5711,5712,495003,Politeknik STIA LAN Makassar,Sulawesi Selatan,Negeri,Aktif,45,550,1 : 12.2,45,805,1 : 17.9
5712,5713,504001,Akademi Metrologi dan Instrumentasi,Jawa Barat,None,Aktif,9,165,1 : 18.3,9,165,1 : 18.3
5713,5714,513001,Sekolah Tinggi Intelijen Negara,Jawa Barat,None,Aktif,22,0,1 : 0,22,0,1 : 0
5714,5715,525001,Politeknik Pekerjaan Umum,Jawa Tengah,None,Aktif,23,0,1 : 0,23,137,1 : 6


# Exploratory Data Analysis

In [22]:
nominatim = Nominatim(user_agent='id-state')
photon = Photon(user_agent='id_state')

In [69]:
def get_location(place):
    loc = photon.geocode(place)
    try:
        return loc.latitude, loc.longitude
    except:
        return None

### Peta Distribusi Jumlah Perguruan Tinggi di Tiap Provinsi

In [25]:
byProv = data.groupby('Provinsi').count()
byProv['Location'] = [get_location(prov) for prov in byProv.index]
byProv.Location

Provinsi
Aceh                                     (4.3685491, 97.0253024)
Bali                            (-8.23565805, 115.1223865976744)
Bangka Belitung                        (-2.7052886, 106.3585607)
Banten                         (-6.47800285, 105.54102849016589)
Bengkulu                               (-3.5186763, 102.5359834)
D.I. Yogyakarta        (-7.9778383999999996, 110.36722565020224)
D.K.I. Jakarta                          (-6.1753942, 106.827183)
Gorontalo                               (0.7186174, 122.4555927)
Jambi                           (-1.6454854, 102.93517597904014)
Jawa Barat                             (-6.8891904, 107.6404716)
Jawa Tengah                     (-5.6259648, 110.37164897312938)
Jawa Timur                             (-7.6977397, 112.4914199)
Kalimantan Barat       (-1.2003376000000001, 109.93193384532759)
Kalimantan Selatan                     (-2.9285686, 115.3700718)
Kalimantan Tengah                       (-1.499583, 113.2903307)
Kalimantan Timur

In [72]:
m = folium.Map(location=get_location('Indonesia'), zoom_start=5, tiles='CartoDB dark_matter', min_zoom=5, max_zoom=5)
for prov in byProv.index:
    folium.CircleMarker(location=byProv.loc[prov, 'Location'], 
                        popup=f'''<strong>{prov}</strong>\
                        <br><br>\
                        {byProv.loc[prov, 'Kode PT']}''',
                        radius=byProv.loc[prov, 'Kode PT']/10,
                        stroke=False, fill=True, fill_opacity=0.5
                       ).add_to(m)

m.save('pt_per_prov.html')
m

### Peta Distribusi Jumlah Mahasiswa di Tiap Provinsi - 2019

### Peta Distribusi Jumlah Dosen di Tiap Provinsi - 2019

### Peta Distribusi Mahasiswa - 2019

### Peta Distribusi Dosen - 2019

### Peta Rasio Dosen dan Mahasiswa - 2019